In [1]:
import cassandra
from cassandra.cluster import Cluster, ConnectionException

In [2]:
nodes = ['127.0.0.1'] # list of cluster devices

In [5]:
def getClusterSession(nodes):
    try:
        cluster = Cluster(nodes)
        
        session = cluster.connect()
        
        print('Connection established successfully.')
        return session
        
    except(ConnectionException, Exception) as exception :
        print ("Error while connecting to Cassandra Cluster\n", exception)
            

def closeSession(session):
    #closing database connection.
    if(session):
        session.shutdown()
        print("Cassandra connection is closed")

In [6]:
session = getClusterSession(nodes)
closeSession(session)

Connection established successfully.
Cassandra connection is closed


In [ ]:
def createKeyspace(conString):
    command = '''
                CREATE KEYSPACE IF NOT EXISTS Employees
                WITH REPLICATION = 
                {'class': 'SimpleStrategy', 'replication_factor' : 1}
                '''
    conn = getDBConnection(conString)
    cursor = conn.cursor()
    
    try:
        
        cursor.execute(command)
        print('Table Successfully Created.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while creating table.", error)
    finally:
        cursor.close()
        closeConnection(conn)
    

In [ ]:
def createEmployeeTable(conString):
    command = '''
                CREATE TABLE IF NOT EXISTS Employee(
                    EmployeeID serial PRIMARY KEY,
                    EmployeeName varchar(50) NOT NULL,
                    ManagerID int,
                    FOREIGN KEY (ManagerID) REFERENCES Employee(EmployeeID)
                    ON DELETE SET NULL
                );'''
    conn = getDBConnection(conString)
    cursor = conn.cursor()
    
    try:
        
        cursor.execute(command)
        print('Table Successfully Created.')
        
    except(Exception, psycopg2.Error) as error :
        print ("Error while creating table.", error)
    finally:
        cursor.close()
        closeConnection(conn)
    